In [ ]:
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import os
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

In [ ]:
def convert_df_to_numpy(df, scaler):
  y = df['label'].to_numpy().astype(np.int32)
  X = scaler.transform(df.drop(columns=['label']).to_numpy())
  return X, y

center_surround_train_df = pd.read_csv("./center_surround_train.csv")
center_surround_test_df = pd.read_csv("./center_surround_test.csv")
center_surround_valid_df = pd.read_csv("./center_surround_valid.csv")

center_surround_scaler = StandardScaler()
center_surround_scaler.fit(center_surround_train_df.drop(columns=['label']).to_numpy())

center_surround_train_data = convert_df_to_numpy(center_surround_train_df, center_surround_scaler)
center_surround_test_data = convert_df_to_numpy(center_surround_test_df, center_surround_scaler)
center_surround_valid_data = convert_df_to_numpy(center_surround_valid_df, center_surround_scaler)

print(center_surround_train_data[0][:5])
print(center_surround_train_data[1][:5])

In [ ]:
INPUT_DIM = 2
OUTPUT_DIM = 2
HIDDEN_NODES = 4
LEARNING_RATE = 0.01

In [44]:
class FeedForwardNeuralNetwork:
  def __init__(self, train_data, test_data, valid_data, input_dim, output_dim, hidden_nodes, learning_rate):
    self.train_data = train_data
    self.test_data = test_data
    self.valid_data = valid_data
    self.input_dim = input_dim
    self.output_dim = output_dim
    self.hidden_nodes = hidden_nodes
    self.learning_rate = learning_rate

    self.initialize_weights_and_biases()

  def initialize_weights_and_biases(self):
    """
    Initialize the weights and biases between the 
    input and hidden layer and then the hidden layer
    to the output layer. 

    The rows correspond to the number of input dimensions
    and the columns correspond to the number of output dimensions.

    Example:
    (2, 4) between the input layer and the output layer
    refers to 2 dimensions of [x1, x2] from the input layer row-wise,
    and 4 refers to the number of hidden nodes in the next layer column-wise.
    """    
    self.W_input_hidden = np.random.random((self.input_dim, self.hidden_nodes))
    self.B_input_hidden = np.zeros((self.hidden_nodes,))

    self.W_hidden_output = np.random.random((self.hidden_nodes, self.output_dim))
    self.B_hidden_output = np.zeros((self.output_dim))

    print("Input layer --> Hidden Layer")
    print(self.W_input_hidden.shape, self.B_input_hidden.shape)
    print(self.W_input_hidden)
    print()
    print("Hidden Layer --> Output Layer")
    print(self.W_hidden_output.shape, self.B_hidden_output.shape)
    print(self.W_hidden_output)
    return


FeedForwardNeuralNetwork(
  train_data=center_surround_train_data,
  test_data=center_surround_test_data,
  valid_data=center_surround_valid_data,
  input_dim=INPUT_DIM,
  output_dim=OUTPUT_DIM,
  hidden_nodes=HIDDEN_NODES,
  learning_rate=LEARNING_RATE
)

Input layer --> Hidden Layer
(2, 4) (4,)
[[0.80007097 0.20403563 0.55508495 0.7330713 ]
 [0.61598545 0.18802473 0.35538457 0.78379178]]

Hidden Layer --> Output Layer
(4, 2) (2,)
[[0.55422651 0.00522961]
 [0.76099076 0.03531135]
 [0.74573378 0.20248056]
 [0.95807348 0.36794075]]
